**Lambda Function code in AWS** 

In [ ]:
import json
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime, date, timedelta
from pytz import timezone

def lambda_handler(event, context):
    # TODO implement
    schema="gans_database"  
    host="wbs-project3-db.ccxgm0tmlzuc.us-east-1.rds.amazonaws.com"        
    user="admin"
    password="Sharp9860"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    
    population_data(con)
    airports_data(con)
    tomorrows_flight_arrivals(con)
    get_weather_data(con)
    
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }
    
def tomorrows_flight_arrivals(con):

  today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
  tomorrow = (today + timedelta(days=1))

  list_for_df = []
  icao_list = ['EDDB']
  for icao in icao_list:
    times = [["00:00","11:59"],["12:00","23:59"]]
    
    for time in times:
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
      querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
      headers = {
          "X-RapidAPI-Key": "0dc165bb4fmsh6e66b9b050bfa2dp1fb4afjsnfb0f7c4c9dc9",
          "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
          }
      response = requests.request("GET", url, headers=headers, params=querystring)
      flights_json = response.json()

      for flight in flights_json['arrivals']:
        flights_dict = {}
        flights_dict['flight_number'] = flight.get('number', None)
        flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
        flights_dict['arrival_icao'] = icao
        flights_dict['arrival_time'] = flight['arrival'].get('scheduledTimeLocal', None)
        list_for_df.append(flights_dict)
        flight_data=pd.DataFrame( list_for_df)
        flight_data['arrival_time']=pd.to_datetime(flight_data['arrival_time']).dt.tz_convert(None)
        
      flight_data.to_sql('flights',if_exists='append',con=con,index=False)
      
      
      
def get_weather_data(con):
    
    OWM_key = 'c4c94a570e01185b8c5aca7295cbafd0'
    data = data = [[1, 'Berlin', 'DE'], [2, 'Hamburg', 'DE'], [3, 'London','GB'], [4, 'Manchester','GB'], [5,'Munich','DE'],[6, 'Stuttgart','DE']]
    cities= pd.DataFrame(data, columns=['city_id', 'city', 'country_code'])
    
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={cities['city'][0]},{cities['country_code'][0]}&appid={OWM_key}&units=metric"
    response = requests.get(url)
    
    weather_dict = {'city_id': [],
                'forecast_time': [],
                'outlook': [],
                'temperature': [],
                'feels_like': [],
                'wind_speed': [],
                'pop': []}
     # iterating over items in 'list' node and selecting the subnodes gives us the exact info we need 
    for hour in response.json()['list']:
        weather_dict['city_id'].append(cities['city_id'][0])
        weather_dict['temperature'].append(hour['main']['temp'])
        weather_dict['wind_speed'].append(hour['wind']['speed'])
        weather_dict['forecast_time'].append(hour['dt_txt'])
        weather_dict['outlook'].append(hour['weather'][0]['description'])
        weather_dict['feels_like'].append(hour['main']['feels_like'])
        weather_dict['pop'].append(hour['pop'])
        weather_df = pd.DataFrame(weather_dict)
    
    weather_df.to_sql('weathers',if_exists='append',con=con,index=False) 
    
    
    
def population_data(con):
  
  url = "https://en.wikipedia.org/wiki/Stuttgart"
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")
  response_dict = {}
  list_for_df=[]
  response_dict['city'] = soup.select(".firstHeading")[0].get_text()
  response_dict['country'] = soup.select(".infobox-data")[0].get_text()
  response_dict['population'] = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
  list_for_df.append(response_dict)
  cities_df = pd.DataFrame(list_for_df)
  cities_df['population']= cities_df['population'].str.replace(',','')
  cities_df.to_sql('cities',if_exists='append',con=con,index=False)
  return cities_df


def airports_data(con):
  
  airport = {'city_id':[1,1,2,3,4,5,5,5,6,6,6],'airport_icao':['EDDB','EDDT','EDDH','EGMC','EGSH','EDDM','EDJA','LOWI','EDDS','EDSB','EDNY'],'airport_name':['Berlin BrandenBurg Airport','Berlin Tegel Otto Linienthal Airport','Hamburg Helmut Schmidt Airport','Southend Airport','Norwich International Airport','Munich Airport','Memmingen Allgau Airport','Innsbruck Airport','Stuttgart Airport','Karlsruhe Baden Baden Airport','Friedrichschafen Airport']}
  air_test = pd.DataFrame.from_dict(airport)
  air_test.to_sql('airports',if_exists='append',con=con,index=False)
  return air_test

  